## Scraping vs Crawling

extracting data from anywhere(include web) vs refers to downloading pages from the web

any scale vs Large Scale

Deduplication : no need vs need

Requirements : crawl agent and parser vs crawl agent

                  Internet

                     ↓

Scheduler ->(URLs) Downloader/parser -> DB and File Storage

In [1]:
# https://validator.w3.org

In [1]:
from bs4 import BeautifulSoup
import re
from html import unescape
import requests
headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}


def download(method, url, headers = headers, params = param, data = None, timeout = 1, maxretries = 3):

    try:        
        resp = requests.request(method, url, params = param, data = data, headers = headers)
        resp.raise_for_status()
    
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxretries > 0:
            print(maxretries)
            time.sleep(timeout)
            resp = download(method, url, headers, param, data, timeout, maxretries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)

    return resp

NameError: name 'param' is not defined

In [43]:
url="http://www.ppomppu.co.kr/zboard/zboard.php" #?id=ppomppu
param={"id":"ppomppu"}
html = download("get", url, param)

In [40]:
dom = BeautifulSoup(html.text,"lxml")

In [44]:
for tr in dom.select("tr[class^=list]")[1:]:
    td = tr.find_all("td", recursive=False)
    print(requests.compat.urljoin(url,td[3].a.img["src"]))
    print(td[3].select("td[valign=middle] > a")[0].text)
    temp = td[5].text.split("-")
    print((0,0) if len(temp) < 2 else ",".join(temp))
    print(td[6].text.strip())
    print()

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/1/small_323881.jpg?t=20190716150504
[11번가] LG 공식판매점 디오스 인덕션 전기레인지 BEF3G 3구 (1,490,000/무료)
0 , 1
331

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/0/small_323880.jpg?t=20190716150035
[셀티바] 다이어트유산균(48%할인/무료배송)
(0, 0)
254

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/9/small_323879.jpg?t=20190716145420
[위메프] EVA 30CM 마사지 폼롤러 (5900/무료)
(0, 0)
1558

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/8/small_323878.jpg?t=20190716145350
[위메프] LG 미니빔 HF60LA FHD 1400루멘 (835,640/무료)
(0, 0)
739

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/7/small_323877.jpg?t=20190716144040
[11번가] 리스테린 그린티 마일드 750mlx2+250mlx2  (13,500/무료배송)
(0, 0)
420

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/6/small_323876.jpg?t=20190716134948
[Gsuite] Gsuite Basic India (3700원(월) / 무료)
3 , 0
5339

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/5/small_323875.jpg?t=20190716133713
[하이마트몰] 갤럭시버즈 (138,500원 / 무배) 오너스 첫달 무료체험시, 최대 50% 페이백
14 , 

In [48]:
#Focused Crawling
def parseURL(seed):
    urlList = list()
    
    html = download("get", seed[0])
    dom = BeautifulSoup(html.text, "lxml")
    
    
    return [(requests.compat.urljoin(seed[0], _["href"]), seed[1]+1)
            for _ in dom.find_all("a") 
            if seed[1] < 2 and _.has_attr("href") and len(_["href"]) > 3 
               and checkBlog(requests.compat.urljoin(seed[0],_["href"]))]

In [45]:
# Focused Crawling
url = "http://www.ppomppu.co.kr/zboard/zboard.php"
param={"id":"freeboard"}
html = download("get",url,param)
dom=BeautifulSoup(html.text,"lxml")

In [46]:
dom

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="PPOMPPU CO." name="author"/>
<meta content="뽐뿌" name="description"/>
<meta content="" name="keywords"/>
<script async="async" src="//www.googletagservices.com/tag/js/gpt.js"></script>
<script>var googletag = googletag || {};googletag.cmd = googletag.cmd || [];</script>
<!--
<meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, user-scalable=yes, target-densitydpi=device-dpi" />
-->
<meta content="http://www.ppomppu.co.kr/images/icon_app_20160427.png" property="og:image"/>
<meta content="뽐뿌" property="og:site_name"/>
<title>뽐뿌 - 자유게시판</title><!--<link href='h

In [50]:
def checkBlog(url):
    return requests.compat.urlparse(url)[1] == 'www.ppomppu.co.kr' 

In [86]:
dom.select(".list_vspace a")

AttributeError: 'list' object has no attribute 'find_parent'

In [100]:
def parseURL(seed):
    html = download("get",seed)
    dom = BeautifulSoup(html.text,"lxml")
    
    return [requests.compat.urljoin(seed, _.find_parent()["href"]) for _ in dom.select("font.list_title")
           if _.find_parent() and _.find_parent().has_attr("href")]

In [101]:
def parseContent(url):
    html = download("get",seed)
    dom = BeautifulSoup(html.text,"lxml")
    
    return {"title":dom.select_one("font.view_title2").text.strip(),
           "body":dom.select_one("td.board-contents").text.strip(),
           "comments":[_.text.strip()
                      for _ in dom.select("div[id^commentContent_]")]}

In [102]:
url="http://www.ppomppu.co.kr/zboard/zboard.php" #?id=ppomppu
param={"id":"freeboard"}
html = download("get", url, param)
dom = BeautifulSoup(html.text,"lxml")

seed=list()
seed.extend([requests.compat.urljoin(url,_["href"]) for _ in dom.select("#page_list a")])
seen = list()
queue = list() # 1은 depth를 뜻함 
             
while seed:
    baseURL = seed.pop(0); # 무조건 기본 주소 올거고 -> 들어있는애가 걔밖에 없으니까
    seen.append(baseURL) # 방문한 곳 저장
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue : {0}, Seen : {1}".format(len(queue), len(seen)))

Queue : 30, Seen : 1
Queue : 60, Seen : 2
Queue : 90, Seen : 3
Queue : 120, Seen : 4
Queue : 150, Seen : 5
Queue : 180, Seen : 6
Queue : 210, Seen : 7
Queue : 240, Seen : 8
Queue : 270, Seen : 9
Queue : 300, Seen : 10


In [103]:
import sqlite3

con = sqlite3.connect('bot.db')
cur = con.cursor()

In [104]:
cur.executescript("""
    DROP TABLE IF EXISTS table3;
    CREATE TABLE table3 (
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        title TEXT NOT NULL,
        content TEXT NOT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
    
    DROP TABLE IF EXISTS table4;
    CREATE TABLE table4 (
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        table3_id INTEGER NOT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
""")
con.commit()

# DROP Table IF EXISTS table1/2;

In [105]:
while queue:
    baseURL = queue.pop(0)
    seen.append(baseURL)
    content = parseContent(baseURL)
    cur.execute("""
        INSERT INTO table3(title, content) VALUES(?,?)
    """, [content["title"], content["body"]])
    con.commit()
    
    cur.execute("""
        SELECT id
        FROM table3
        WHERE title=? AND content=?;
    """, [content["title"], content["body"]])
    id = cur.fetchone()[0]
    
    for comment in content["comments"]:
        cur.execute("""
            INSERT INTO table4(table3_id, comment) VALUES(?,?)            
        """, [table3_id, comment])
    con.commit()

MissingSchema: Invalid URL '[]': No schema supplied. Perhaps you meant http://[]?

NameError: name 'driver' is not defined